In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio
from pprint import pprint

D:\Projects\venv2\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
D:\Projects\venv2\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
D:\Projects\venv2\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


General settings and functions

In [2]:
LANGUAGE = "fa"
MODEL = "openai/whisper-medium"
FINETUNED_MODEL_PATH = r"D:\Git_repos\Persian_text_correction\models\finetuned_whisper-large-fa_1\checkpoint-5200"

In [3]:
processor = WhisperProcessor.from_pretrained(MODEL, language=LANGUAGE, task="transcribe")

# --------- Original Whisper-medium ---------

model = WhisperForConditionalGeneration.from_pretrained(MODEL)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language=LANGUAGE, task="transcribe",)

model.eval().to("cuda")

# -------- Finetuned Whisper-medium ---------
model_path = r"D:\Git_repos\Persian_text_correction\models\finetuned_whisper-large-fa_1\training_history\2000 steps - train ALL val 400\checkpoint-2000"
finetuned_model = WhisperForConditionalGeneration.from_pretrained(model_path)
finetuned_model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language=LANGUAGE, task="transcribe")

finetuned_model.eval().to("cuda")

D:\Projects\venv2\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1024)
      (layers): ModuleList(
        (0-23): 24 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=Tru

In [4]:
def transcribe(audio_path, model, processor):
    waveform, sample_rate = torchaudio.load(audio_path)
    if sample_rate != 16000:
        waveform = torchaudio.functional.resample(waveform, orig_freq=sample_rate, new_freq=16000)

    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    inputs = processor(
        waveform.squeeze().numpy(), 
        sampling_rate=16000,
        return_tensors="pt"
    ).input_features.to("cuda")

    generated_ids = model.generate(inputs)
    return processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Input audio file

In [17]:
audio_file= r"D:\Datasets\Audio_datasets\myaudio_tiny\myaudio\12270012.wav"
# audio_file= r"D:\Datasets\Audio_datasets\myaudio_tiny\myaudio\12270029.wav"
# audio_file= r"D:\Datasets\Audio_datasets\myaudio_tiny\myaudio\12270036.wav"
# audio_file= r"D:\Datasets\Audio_datasets\myaudio_tiny\myaudio\12270038.wav"
# audio_file= r"D:\Datasets\Audio_datasets\myaudio_tiny\myaudio\12560242.wav"

Run inference on original model

In [18]:
res = transcribe(audio_file, model, processor)

print("Original model output:")
pprint(res)

Original model output:
(' از سب تا شب به خاطر اون کار می کرد و خودش رو از پامین داخد او هر روز خودش '
 'رو به آب آتیش می زد بلکه')


Run inference on finetuned model

In [19]:
res_finetuned = transcribe(audio_file, finetuned_model, processor)

print("Finetuned model output:")
pprint(res_finetuned)

Finetuned model output:
('از صبح تا شب به خاطر اون کار میکرد و خودش را از پامین داخت. او هر روز خودش '
 'را به آب آتیش میزد بلکه')
